In [1]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

/opt/anaconda3/envs/torch-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Install missing langchain-community package
%pip install langchain-community

# Correct imports for newer LangChain versions
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv(override=True)

Note: you may need to restart the kernel to use updated packages.


True

In [3]:
model="gpt-3.5-turbo"
db_name="vector_db"

In [4]:
load_dotenv(override=True)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai=OpenAI()


In [5]:
#for checking the current working environment
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

Python executable: /opt/anaconda3/envs/torch-env/bin/python
Python version: 3.10.18 (main, Jun  5 2025, 08:13:51) [Clang 14.0.6 ]


In [6]:
# Run this in your notebook cell
import subprocess
import sys

# Install pypdf in current environment
subprocess.check_call([sys.executable, "-m", "pip", "install", "pypdf"])

0

In [7]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf_files_one_by_one():
    base_folder = Path('../data/documents')
    all_documents = []
    loaded_files = []
    failed_files = []
    
    # Find all PDF files recursively
    pdf_files = list(base_folder.rglob("*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process:")
    print("-" * 50)
    
    for i, pdf_file in enumerate(pdf_files, 1):
        print(f"\n[{i}/{len(pdf_files)}] Processing: {pdf_file.name}")
        print(f"Location: {pdf_file}")
        
        try:
            # Load individual PDF
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add metadata about source file
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_path'] = str(pdf_file)
            
            all_documents.extend(documents)
            loaded_files.append({
                'file': pdf_file.name,
                'pages': len(documents),
                'status': 'SUCCESS'
            })
            
            print(f"✓ SUCCESS: Loaded {len(documents)} pages")
            
        except Exception as e:
            failed_files.append({
                'file': pdf_file.name,
                'error': str(e),
                'status': 'FAILED'
            })
            print(f"✗ FAILED: {e}")
    
    # Summary report
    print("\n" + "="*60)
    print("LOADING SUMMARY")
    print("="*60)
    print(f"Total files found: {len(pdf_files)}")
    print(f"Successfully loaded: {len(loaded_files)}")
    print(f"Failed to load: {len(failed_files)}")
    print(f"Total pages extracted: {len(all_documents)}")
    
    # Detailed success report
    if loaded_files:
        print(f"\n✓ SUCCESSFUL FILES:")
        for file_info in loaded_files:
            print(f"  - {file_info['file']}: {file_info['pages']} pages")
    
    # Detailed failure report
    if failed_files:
        print(f"\n✗ FAILED FILES:")
        for file_info in failed_files:
            print(f"  - {file_info['file']}: {file_info['error']}")
    
    return all_documents, loaded_files, failed_files

# Execute the loading
print("Starting PDF loading process...")
documents, success_list, failure_list = load_pdf_files_one_by_one()

# Optional: Show sample content from first document
if documents:
    print(f"\n📄 SAMPLE CONTENT from {documents[0].metadata.get('source_file', 'Unknown')}:")
    print("-" * 50)
    print(documents[0].page_content[:300] + "...")

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)


Starting PDF loading process...
Found 7 PDF files to process:
--------------------------------------------------

[1/7] Processing: SJ_Developers.pdf
Location: ../data/documents/SJ DEVELOPERS DATA/SJ_Developers.pdf


Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 78 0 (offset 0)
Ignoring wrong pointing object 97 0 (offset 0)
Ignoring wrong pointing object 99 0 (offset 0)


✓ SUCCESS: Loaded 9 pages

[2/7] Processing: Sri Sathya Sai Serenity_Layout_Final.pdf
Location: ../data/documents/SJ DEVELOPERS DATA/Sri Sathya Sai Serenity_Layout_Final.pdf
✓ SUCCESS: Loaded 1 pages

[3/7] Processing: AmariFarms_Brochure.pdf
Location: ../data/documents/SJ DEVELOPERS DATA/AmariFarms_Brochure.pdf
✓ SUCCESS: Loaded 30 pages

[4/7] Processing: VILLA 01.pdf
Location: ../data/documents/SJ DEVELOPERS DATA/VILLA 01.pdf
✓ SUCCESS: Loaded 2 pages

[5/7] Processing: VILLA 02.pdf
Location: ../data/documents/SJ DEVELOPERS DATA/VILLA 02.pdf
✓ SUCCESS: Loaded 2 pages

[6/7] Processing: REGISTRATION OF FIRMS.pdf
Location: ../data/documents/SJ DEVELOPERS DATA/REGISTRATION OF FIRMS.pdf
✓ SUCCESS: Loaded 1 pages

[7/7] Processing: CIBW1022010.pdf
Location: ../data/documents/SJ DEVELOPERS DATA/CIBW1022010.pdf
✓ SUCCESS: Loaded 17 pages

LOADING SUMMARY
Total files found: 7
Successfully loaded: 7
Failed to load: 0
Total pages extracted: 62

✓ SUCCESSFUL FILES:
  - SJ_Developers.pdf: 9 pag